# Data Drift Dashboard
Here we will go back to using EvidentlyAI to create a proof of concept data drift dashboard. We will use the full training data as reference and the full five validation cohorts as analysis for this example.

In [1]:
import pickle
import numpy as np
import re
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt;
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import os
import sys
import gcsfs
sys.path.append(os.path.abspath("/home/jupyter/POM-feature-drift"))

In [36]:
import nannyml as nml
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset
from evidently import ColumnMapping

In [5]:
# Full reference data, will need to generate different reports for different treatments as the models are trained on different data.
ref = pd.read_csv('gs://andrew-pom/Revenue/ta_step_up_v2_clean.csv', low_memory = False)

In [15]:
ref.treatment_type.value_counts()

NT    455582
L      71951
M      71487
H      71466
Name: treatment_type, dtype: int64

## LOW TA
Just doing the one treatment as a test, will look more into how to run it for the different treatments 

In [17]:
analysis = pd.read_csv('data/ta_low_reference_full.csv', low_memory = False)

In [44]:
analysis.timestamp = pd.to_datetime(analysis.timestamp)
analysis.sort_values(by = 'timestamp', inplace = True)

In [19]:
with open('pickle_files/DTV_UK_ta_L.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)
features = model.feature_name_

In [34]:
low_ref = ref[ref.treatment_type == 'L']
dates = []
for index in low_ref.ind:
    dates.append(index.split('-')[0])

timestamps = []
for i, date in enumerate(dates):
    timestamps.append(pd.to_datetime(str(dates[i]) + '-0', format = '%Y%W-%w'))
low_ref['timestamp'] = timestamps
low_ref = low_ref[features + ['timestamp']]


In [45]:
low_ref.sort_values(by = 'timestamp', inplace = True)

In [37]:
# column_mapping = ColumnMapping()
# column_mapping.datetime = 'timestamp'

In [49]:
drift_report = Report(metrics=[DataDriftPreset()])

SyntaxError: invalid syntax (4179803666.py, line 1)

In [47]:
drift_report.run(reference_data = low_ref, current_data = analysis[features])

In [48]:
drift_report.save_html('drift_reports/ta_low.html')

In [51]:
drift_report.save_json('drift_reports/ta_low.json')